How to run a code with stopping conditions
====================

Stopping conditions should be considered interrupts that are initiated through a special conditions in the running code. Topping conditions leave the running code in a restartable state while existing the code to hand the control back to the python (i.e. the AMUSE) script. There are stopping conditions in most codes, but here we only discuss one of them in the stellar dynamics codes, which is called: *collision_detection*

For reference you can read chapter 2 of Portegies Zwart & McMillan 2018 (2018araa.book.....P). 

With this tutorial you will learn
- how to generate inital conditions (mass-function, King-model)
- to initialize an gravitational N-body code
- channels and intra-code data transfer
- detecting binaries
- plotting results
- making cumulative distributions

In [ ]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from amuse.units import units

Let's start by generating the initial conditions and initializing the N-body code.

In [ ]:
n_stars = 100
alpha_IMF = -2.35

from amuse.lab import new_powerlaw_mass_distribution
m_stars = new_powerlaw_mass_distribution(n_stars, 
                                         10.0|units.MSun, 
                                         100.0|units.MSun, 
                                         alpha_IMF)

r_cluster = 1.0 | units.parsec
from amuse.units import nbody_system
converter=nbody_system.nbody_to_si(m_stars.sum(), r_cluster)
from amuse.ic.plummer import new_plummer_model
stars=new_plummer_model(n_stars, convert_nbody=converter)
stars.mass = m_stars
setattr(stars, "collision_radius", 0|units.RSun)
stars.scale_to_standard(converter)

from amuse.community.ph4.interface import ph4
from amuse.community.seba.interface import SeBa
from amuse.ext.LagrangianRadii import LagrangianRadii

stellar = SeBa()
stellar.particles.add_particles(stars)

gravity = ph4(converter, number_of_workers=6)
gravity.particles.add_particles(stars)

Now let's declare the channels.
This is somewhat more elaborate here, because there are two new parameters to declare, which are the *relative_mass* and the *collision_radius*
Both parameters are needed in order to assure that our stellar collisions behave as expected

In [ ]:
stellar_attributes = ["mass", "radius", "age", "temperature", "luminosity", "relative_mass"]
channel = {"from_stellar":
              stellar.particles.new_channel_to(stars,
              attributes=stellar_attributes,
              target_names=stellar_attributes),
            "from_gravity": 
              gravity.particles.new_channel_to(stars,
              attributes=["x", "y", "z", "vx", "vy", "vz", "mass"],
              target_names=["x", "y", "z", "vx", "vy", "vz", "mass"]),
            "to_gravity": 
              stars.new_channel_to(gravity.particles,
              attributes=["mass", "collision_radius"],
              target_names=["mass", "radius"])}
    
channel["from_stellar"].copy()

We here explicitely copied particle attributes which we think are of interesting for this experiment, including stellar mass, radius, age, temperature and luminosity.

##### *collision_radius*
An additional parameter was introduced to keep track of the collisional radius for the stars. Stars tend to be small compared to the inter-stellar distance. In order to warrant some nice collisions in our simulation we may want to expand the stellar size in comparison with the equilibrium photospheric radius of the stars. We can do this by adding an additional parameter, here called *collision_radius* to the particle set. Notive that the *collision_radius* is copied into the attribute *radius* of the stars in the N-body code, because there is no separate collision-radius in the N-body code, but the attribute *radius* is used to check for two objects to come close before an exception is thrown.

##### *relative_mass*

Let's check if this is what we expected.

In [ ]:
def plot_snapshot(bodies):
    v = (bodies.vx**2 + bodies.vy**2 + bodies.vz**2).sqrt()
    s = bodies.mass.value_in(units.MSun)
    fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(11, 3))
    ax1.scatter(stars.temperature.value_in(units.K),
                stars.luminosity.value_in(units.LSun), 
                c=v.value_in(units.kms),
                s=s)
    ax1.set_xlim(6.e+4, 20000)
    ax1.set_ylim(1.e+3, 1.e+7)
    ax1.loglog()
    ax1.set_xlabel("T [K]")
    ax1.set_ylabel("L [$L_\odot$]")
    ax2.scatter(bodies.x.value_in(units.pc), 
                bodies.y.value_in(units.pc), 
                c=v.value_in(units.kms), 
                s=s)
    pyplot.gca().set_aspect('equal', adjustable='box')
    ax2.set_xlabel("x [pc]")
    ax2.set_ylabel("y [pc]")
    ax2.set_xlim(-5, 5)
    ax2.set_ylim(-5, 5)
    pyplot.show()
plot_snapshot(stars)

Now initiate the stopping conditions in order to detect a merger between two stars.

In [ ]:
stopping_condition = gravity.stopping_conditions.collision_detection
stopping_condition.enable()
collision_radius_multiplication_factor = 1000

We introduced a multiplication factor in order to acquire many nice collisions.

In [ ]:
def merge_two_stars(bodies, particles_in_encounter):
    print("Actually perform the merger between:")
    print(particles_in_encounter)
    com_pos = particles_in_encounter.center_of_mass()
    com_vel = particles_in_encounter.center_of_mass_velocity()
    new_particle=Particles(1)
    new_particle.mass = particles_in_encounter.total_mass()
    new_particle.age = min(particles_in_encounter.age) \
                         * max(particles_in_encounter.mass)/new_particle.mass
    new_particle.position = com_pos
    new_particle.velocity = com_vel
    new_particle.radius = particles_in_encounter.radius.sum()
    bodies.add_particles(new_particle)
    print("Two stars (M=",particles_in_encounter.mass.in_(units.MSun),
          ") collided with d=", com_pos.length().in_(units.au))
    bodies.remove_particles(particles_in_encounter)

In [ ]:
from amuse.lab import Particles
def resolve_collision(collision_detection, gravity, stellar, bodies):
    if collision_detection.is_set():
        print("Well, we have an actual collision between two or more stars.")
        print("This happened at time=", stellar.model_time.in_(units.Myr))
        for ci in range(len(collision_detection.particles(0))): 
            encountering_particles = Particles(particles=[collision_detection.particles(0)[ci],
                                                      collision_detection.particles(1)[ci]])
            colliding_stars = encountering_particles.get_intersecting_subset_in(bodies)
            merge_two_stars(bodies, colliding_stars)
            bodies.synchronize_to(gravity.particles)
            bodies.synchronize_to(stellar.particles)

In [ ]:
end_time = 10.0 | units.Myr
model_time = 0 | units.Myr
rvir = [] | units.pc
rcore = [] | units.pc
cradius = 0 | units.pc
time = [] | units.Myr
t_diag = 1| units.Myr
while(model_time<end_time):
    dt = stellar.particles.time_step.min()
    model_time += dt
    stellar.evolve_model(model_time)
    channel["from_stellar"].copy()
    stars.collision_radius = stars.radius * collision_radius_multiplication_factor
    channel["to_gravity"].copy()
    gravity.evolve_model(model_time)
    resolve_collision(stopping_condition, gravity, stellar, stars)
    channel["from_gravity"].copy()
    
    if model_time >= t_diag:
        t_diag += 1 | units.Myr

        print("Evolved to t=", stellar.model_time.in_(units.Myr), 
              gravity.model_time.in_(units.Myr),
              "N=", len(stars),
              "mass=", stars.mass.sum().in_(units.MSun), 
              "rvir=", stars.virial_radius().in_(units.pc))
        cpos, cradius, cdensity = stars.densitycentre_coreradius_coredens(unit_converter=converter)
    rcore.append(cradius)
    rvir.append(stars.virial_radius())
    time.append(model_time)
    
stellar.stop()
gravity.stop()

In [ ]:
plot_snapshot(stars)
pyplot.plot(time.value_in(units.Myr), rvir.value_in(units.parsec))
pyplot.plot(time.value_in(units.Myr), rcore.value_in(units.parsec))
pyplot.show()

You have calculated the evolution of a small cluster of 100 stars. Your calculation included the Newtonian dynamics, stellar evolution and collision.
the collisions were rather simple, sticky spheres, and we have chosen the stellar radii to be a 1000 times larger than normal.

Assignmnets and questions:
---------------

### Question 1:
